In [1]:
import rasterio
from rasterio.fill import fillnodata
from affine import Affine
import pandas as pd
import numpy as np
import climatezones as cz 
%load_ext autoreload
%autoreload 2

In [5]:
points = pd.read_csv('tmp/points.csv')
points = points[~points.long.isnull()]

def convert_points(file, col, points):
    raster = rasterio.open(file)
    transformer = rasterio.transform.AffineTransformer(raster.transform)
    band1 = raster.read(1)
    points['x'], points['y'] = transformer.rowcol(points.long, points.lat)
    fpoints = points[points.x <= raster.height]
    fpoints[col] = band1[fpoints.x, fpoints.y]
    return fpoints, raster, band1


In [ ]:
file = 'dbs/geospatial/globalelu/World_ELU_2015.tif'
col = 'elu'
fpoints, raster, band = convert_points(file, col, points)
fpoints[["gbifid", col]].to_csv("dbs/globalelu.csv", header=True, index=False)

In [ ]:
beck = 'dbs/geospatial/Beck_KG_V1/Beck_KG_V1_present_0p0083.tif'
kgpoints, kg, band = convert_points(beck, 'kg', points)
kgpoints[["gbifid", "kg"]].to_csv("dbs/beck-kg-v1.csv", header=True, index=False)

In [ ]:
fpoints.loc[fpoints.elu == 0]

In [ ]:
mask = raster.read_masks(1)

In [ ]:
mask

In [13]:
zero_mask = mask != 0
result = fillnodata(raster, mask)

ValueError: An mask array is required

In [ ]:
fpoints.groupby(by="elu").count()